In [ ]:
from pybis import Openbis
import json
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Javascript, HTML
from datetime import datetime

In [ ]:
# Global variables
OPENBIS_SESSION = None
RAW_MATERIALS_TYPES = ["CRYSTAL", "WAFER_SUBSTRATE", "2D_LAYERED_MATERIAL", "MOLECULE"]
PROCESS_SAMPLE_TYPES = ["SPUTTERING", "ANNEALING", "DEPOSITION"]
SAMPLES_COLLECTION_OPENBIS_PATH = "/MATERIALS/SAMPLES/SAMPLE_COLLECTION"

In [ ]:
# Widgets
samples_dropdown = widgets.Dropdown(description='Sample', disabled=False, layout = widgets.Layout(width = '400px'))
samples_dropdown.style = {'description_width': '110px'}
sample_details_textbox = widgets.Textarea(value = '', description = '', disabled = True, layout = widgets.Layout(width = '600px', height = '300px'))
sample_metadata_boxes = widgets.HBox([samples_dropdown, sample_details_textbox])

instruments_dropdown = widgets.Dropdown(description='Instrument', disabled=False, layout = widgets.Layout(width = '993px'))
instruments_dropdown.style = {'description_width': '110px'}

experiments_dropdown = widgets.Dropdown(description='Experiment', disabled=False, layout = widgets.Layout(width = '993px'))
experiments_dropdown.style = {'description_width': '110px'}

molecules_dropdown = widgets.Dropdown(description='Molecule', disabled=False, layout = widgets.Layout(width = '350px'))
molecules_dropdown.style = {'description_width': '110px'}
molecule_details_textbox = widgets.Textarea(value = '', description = '', disabled = True, layout = widgets.Layout(width = '415px', height = '250px'))
molecule_image_box = widgets.Image(value = open("images/white_screen.jpg", "rb").read(), format = 'jpg', width = '220px', height = '250px', layout=widgets.Layout(border='solid 1px #cccccc'))
molecule_metadata_boxes = widgets.HBox([molecules_dropdown, molecule_details_textbox, molecule_image_box])

parents_boxes = widgets.VBox([experiments_dropdown, sample_metadata_boxes, instruments_dropdown, molecule_metadata_boxes])

stabilisation_time_value_floatbox = widgets.FloatText(value = 0, description = 'Stabilisation time', disabled = False, layout = widgets.Layout(width = '200px'))
stabilisation_time_value_floatbox.style = {'description_width': '110px'}
stabilisation_time_unit_dropdown = widgets.Dropdown(value = 'sec', description = '', disabled = False, options = ["sec", "min", "hrs"], layout = widgets.Layout(width = '100px'))
stabilisation_time_hbox = widgets.HBox([stabilisation_time_value_floatbox, stabilisation_time_unit_dropdown])

deposition_time_value_floatbox = widgets.FloatText(value = 0, description = 'Deposition time', disabled = False, layout = widgets.Layout(width = '200px'))
deposition_time_value_floatbox.style = {'description_width': '110px'}
deposition_time_unit_dropdown = widgets.Dropdown(value = 'sec', description = '', disabled = False, options = ["sec", "min", "hrs"], layout = widgets.Layout(width = '100px'))
deposition_time_hbox = widgets.HBox([deposition_time_value_floatbox, deposition_time_unit_dropdown])

pressure_value_floatbox = widgets.FloatText(value = 0, description = 'Pressure', disabled = False, layout = widgets.Layout(width = '200px'))
pressure_value_floatbox.style = {'description_width': '110px'}
pressure_unit_dropdown = widgets.Dropdown(value = 'mBar', description = '', disabled = False, options = ["mBar", "Bar", "Pa", "kPa"], layout = widgets.Layout(width = '100px'))
pressure_hbox = widgets.HBox([pressure_value_floatbox, pressure_unit_dropdown])

properties_on_left = widgets.VBox([stabilisation_time_hbox, deposition_time_hbox, pressure_hbox])

substrate_temperature_value_floatbox = widgets.FloatText(value = 0, description = 'Substrate temperature', disabled = False, layout = widgets.Layout(width = '250px'))
substrate_temperature_value_floatbox.style = {'description_width': '150px'}
substrate_temperature_unit_dropdown = widgets.Dropdown(value = 'K', description = '', disabled = False, options = ["K", "oC", "oF"], layout = widgets.Layout(width = '100px'))
substrate_temperature_hbox = widgets.HBox([substrate_temperature_value_floatbox, substrate_temperature_unit_dropdown])

molecule_temperature_value_floatbox = widgets.FloatText(value = 0, description = 'Molecule temperature', disabled = False, layout = widgets.Layout(width = '250px'))
molecule_temperature_value_floatbox.style = {'description_width': '150px'}
molecule_temperature_unit_dropdown = widgets.Dropdown(value = 'K', description = '', disabled = False, options = ["K", "oC", "oF"], layout = widgets.Layout(width = '100px'))
molecule_temperature_hbox = widgets.HBox([molecule_temperature_value_floatbox, molecule_temperature_unit_dropdown])

evaporation_slot_value_intslider = widgets.IntSlider(value = 1, description = 'Evaporation slot', min = 1, max = 6, disabled = False, layout = widgets.Layout(width = '300px'))
evaporation_slot_value_intslider.style = {'description_width': '150px'}
evaporation_slot_details_textbox = widgets.Text(value = '', description = '', placeholder= "Write evaporator slot details...", disabled = False, layout = widgets.Layout(width = '300px'))
evaporation_slot_hbox = widgets.HBox([evaporation_slot_value_intslider, evaporation_slot_details_textbox])

properties_on_right = widgets.VBox([substrate_temperature_hbox, molecule_temperature_hbox, evaporation_slot_hbox])

deposition_properties = widgets.HBox([properties_on_left, properties_on_right])

comments_textbox = widgets.Textarea(value = '', placeholder = "Write comments here...", description = 'Comments', disabled = False, layout = widgets.Layout(width = '993px', height = '200px'))
comments_textbox.style = {'description_width': '110px'}

deposition_name_textbox = widgets.Text(value = '', placeholder = "Write deposition task name here...", description = 'Name', disabled = False, layout = widgets.Layout(width = '400px'))
deposition_name_textbox.style = {'description_width': '110px'}

sample_out_name_textbox = widgets.Text(value = '', placeholder = "Write output sample name here...", description = 'Name', disabled = False, layout = widgets.Layout(width = '400px'))
sample_out_name_textbox.style = {'description_width': '110px'}

create_deposition_button = widgets.Button(description = '', disabled = False, button_style = '', tooltip = 'Save sample', icon = 'save', layout = widgets.Layout(width = '100px', height = '50px'))
quit_button = widgets.Button(description = '', disabled = False, button_style = '', tooltip = 'Main menu', icon = 'home', layout = widgets.Layout(width = '100px', height = '50px'))

bottom_buttons_hbox = widgets.HBox([create_deposition_button, quit_button])

increase_buttons_size = HTML("""
<style>
    .fa-save {
        font-size: 2em !important; /* Increase icon size */
    }
    .fa-home {
        font-size: 2em !important; /* Increase icon size */
    }
</style>
""")

In [ ]:
# Functions
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Function to close the notebook page
def close_notebook(b):
    display(Javascript(f'window.open("home.ipynb", "_blank")'))
    display(Javascript('window.close()'))

def create_deposition_action(b):
    global OPENBIS_SESSION
    
    samples = OPENBIS_SESSION.get_objects(type = "SAMPLE")
    samples_names = [sample.props["$name"] for sample in samples]
    
    if sample_out_name_textbox.value in samples_names:
        display(Javascript(f"alert('{'Sample name already exists!'}')"))
    else:
        if experiments_dropdown.value is None:
            print("Select an experiment.")
        elif samples_dropdown.value is None:
            print("Select a sample.")
        elif instruments_dropdown.value is None:
            print("Select an instrument.")
        elif molecules_dropdown.value is None:
            print("Select a molecule.")
        else:
            sample_parents = [samples_dropdown.value, instruments_dropdown.value, molecules_dropdown.value]
            object_properties = {}
            object_properties["$name"] = deposition_name_textbox.value
            object_properties["stabilisation_time"] = json.dumps({"value": stabilisation_time_value_floatbox.value, "unit": stabilisation_time_unit_dropdown.value})
            object_properties["deposition_time"] = json.dumps({"value": deposition_time_value_floatbox.value, "unit": deposition_time_unit_dropdown.value})
            object_properties["pressure"] = json.dumps({"value": pressure_value_floatbox.value, "unit": pressure_unit_dropdown.value})
            object_properties["substrate_temperature"] = json.dumps({"value": substrate_temperature_value_floatbox.value, "unit": substrate_temperature_unit_dropdown.value})
            object_properties["molecule_temperature"] = json.dumps({"value": molecule_temperature_value_floatbox.value, "unit": molecule_temperature_unit_dropdown.value})
            object_properties["evaporator_slot"] = json.dumps({"evaporator_number": evaporation_slot_value_intslider.value, "details": evaporation_slot_details_textbox.value})
            object_properties["comments"] = comments_textbox.value
            
            deposition_task = OPENBIS_SESSION.new_object(type = "DEPOSITION", 
                                                    collection = experiments_dropdown.value,
                                                    props = object_properties,
                                                    parents = sample_parents)
            deposition_task.save()

            OPENBIS_SESSION.new_object(type = "SAMPLE",
                                    collection = SAMPLES_COLLECTION_OPENBIS_PATH,
                                    props = {"$name": sample_out_name_textbox.value},
                                    parents = [deposition_task]).save()
            print("Upload successful!")

# Function to handle changes in the materials dropdown
def load_molecule_metadata(change):
    global OPENBIS_SESSION
    if molecules_dropdown.value is None:
        molecule_details_textbox.value = ''
        file = open("images/white_screen.jpg", "rb")
        image = file.read()
        molecule_image_box.value = image
    else:
        property_list = [("Name", "$name"), ("IUPAC name", "iupac_name"), ("Sum formula", "sum_formula"), ("SMILES", "smiles"), ("Empa number", "empa_number"), ("Batch", "batch"), ("Amount", "amount"), ("Comments", "comments")]
        
        material_object = OPENBIS_SESSION.get_object(molecules_dropdown.value)
        material_dataset = material_object.get_datasets(type = "ELN_PREVIEW")[0]
        
        if material_dataset is None:
            file = open("images/white_screen.jpg", "rb")
            image = file.read()
            molecule_image_box.value = image
        else:
            material_dataset.download(destination = "images")
            material_dataset_filenames = material_dataset.file_list
            material_image_filepath = material_dataset_filenames[0]
            file = open(f"images/{material_dataset.permId}/{material_image_filepath}", "rb")
            image = file.read()
            molecule_image_box.value = image
        
        material_metadata = material_object.props.all()
        material_metadata_string = ""
        for property in property_list:
            if property[1] in ["amount"]:
                if material_metadata[property[1]] is None:
                    material_metadata_string = f"{material_metadata_string} {property[0]}: {material_metadata[property[1]]}\n"
                else:
                    property_dict = json.loads(material_metadata[property[1]])
                    material_metadata_string = f"{material_metadata_string} {property[0]}: {property_dict['value']} {property_dict['unit']}\n"
            else:
                material_metadata_string = f"{material_metadata_string} {property[0]}: {material_metadata[property[1]]}\n"
        molecule_details_textbox.value = material_metadata_string

def load_sample_metadata(change):
    global OPENBIS_SESSION
    if samples_dropdown.value is None:
        sample_details_textbox.value = ''
        sample_out_name_textbox.value = ''
    else:
        sample_object = OPENBIS_SESSION.get_object(samples_dropdown.value)
        sample_parents_metadata = []
        sample_parents_metadata = get_parents_recursive(sample_object, sample_parents_metadata)
        
        last_sample_process = None
        sample_processes_strings = []
        sample_materials_strings = []
        number_parents = len(sample_parents_metadata)
        parent_idx = 0
        while parent_idx < number_parents:
            parent_metadata = sample_parents_metadata[parent_idx]
            if parent_metadata[0] == "DEPOSITION":
                next_parent_metadata = sample_parents_metadata[parent_idx + 1]
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]}) [{next_parent_metadata[0]} ({next_parent_metadata[3]}, {next_parent_metadata[1]}, {next_parent_metadata[4]})]"
                parent_idx += 1
            else:
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]})"
            
            if parent_metadata[0] in PROCESS_SAMPLE_TYPES:
                sample_processes_strings.append(sample_metadata_string)
                # Get the last sample preparation method performed on the sample in order to search the correct experiment where the sample is being used.
                if last_sample_process is None:
                    last_sample_process = parent_metadata[1]
            else:
                sample_materials_strings.append(sample_metadata_string)
                
            parent_idx += 1
        
        sample_metadata_string = f"PermId: {sample_object.attrs.permId}\nMaterial:"
        
        for material_string in sample_materials_strings:
            sample_metadata_string = f"{sample_metadata_string}\n{material_string}"
            
        sample_metadata_string = f"{sample_metadata_string}\nProcesses:"
        for process_string in sample_processes_strings:
            sample_metadata_string = f"{sample_metadata_string}\n{process_string}"
            
        if last_sample_process is not None:
            last_sample_process_object = OPENBIS_SESSION.get_object(last_sample_process)
            
            # Automatically select the experiment where the last sample process task was saved
            last_sample_process_experiment_id = last_sample_process_object.attrs.experiment
            last_sample_process_experiment = OPENBIS_SESSION.get_experiment(last_sample_process_experiment_id)
            experiments_dropdown.value =  last_sample_process_experiment.permId
            
            # Automatically select the instrument used in the last sample process task
            last_sample_process_object_parents = last_sample_process_object.parents
            for parent in last_sample_process_object_parents:
                parent_object = OPENBIS_SESSION.get_object(parent)
                if parent_object.type == "INSTRUMENT":
                    instruments_dropdown.value = parent_object.permId
        
        sample_details_textbox.value = sample_metadata_string
        sample_out_name_textbox.value = f"{sample_object.props['$name']}_{deposition_name_textbox.value}"

def get_parents_recursive(object, object_parents_metadata):
    if object.attrs.type in PROCESS_SAMPLE_TYPES or object.attrs.type in RAW_MATERIALS_TYPES:
        object_parents_metadata.append([object.attrs.type, object.attrs.permId, object.attrs.registrationDate, 
                                        object.props['$name'], object.props['sum_formula']])
    object_parents = object.parents
    for parent in object_parents:
        parent_object = OPENBIS_SESSION.get_object(parent)
        get_parents_recursive(parent_object, object_parents_metadata)
    return object_parents_metadata

def update_text(change):
    if samples_dropdown.value is not None:
        selected_sample_name = next(label for label, val in samples_dropdown.options if val == samples_dropdown.value)
        sample_out_name_textbox.value = f"{selected_sample_name}_{deposition_name_textbox.value}"


In [ ]:
# Connect with openBIS
session_data = read_json("token.json")
OPENBIS_SESSION = Openbis(session_data["url"], verify_certificates = False)
OPENBIS_SESSION.set_token(session_data["token"])

# Deposition

## Select experiment, sample, and instrument

In [ ]:
experiments = OPENBIS_SESSION.get_collections(type = "EXPERIMENT")
samples = OPENBIS_SESSION.get_objects(type = "SAMPLE")
instruments = OPENBIS_SESSION.get_objects(type = "INSTRUMENT")
molecules = OPENBIS_SESSION.get_objects(type = "MOLECULE")

experiments_names_permids = [(f"{experiment.props['$name']} ({experiment.attrs.identifier})", experiment.permId) for experiment in experiments]
experiments_dates = [datetime.fromisoformat(experiment.registrationDate) for experiment in experiments]
experiments_names_permids = [x for _,x in sorted(zip(experiments_dates, experiments_names_permids), reverse = True)]
experiments_names_permids.insert(0, ('Select an experiment...', None))
experiments_dropdown.options = experiments_names_permids

samples_names_permids = [(sample.props['$name'], sample.permId) for sample in samples]
samples_dates = [datetime.fromisoformat(sample.registrationDate) for sample in samples]
samples_names_permids = [x for _,x in sorted(zip(samples_dates, samples_names_permids), reverse = True)]
samples_names_permids.insert(0, ('Select an input sample...', None))
samples_dropdown.options = samples_names_permids

instruments_names_permids = [(f"{instrument.props['$name']} ({instrument.attrs.permId})", instrument.permId) for instrument in instruments]
instruments_names_permids.insert(0, ('Select an instrument...', None))
instruments_dropdown.options = instruments_names_permids
instruments_dropdown.options = instruments_names_permids

molecules_names_permids = [(f"{molecule.props['$name']} ({molecule.attrs.permId})", molecule.permId) for molecule in molecules]
molecules_empa_numbers = [np.nan if molecule.props['empa_number'] is None else int(molecule.props['empa_number']) for molecule in molecules]
molecules_batches = [np.nan if molecule.props['batch'] is None else molecule.props['batch'] for molecule in molecules]
molecules_names_permids = [x for _, _, x in sorted(zip(molecules_empa_numbers, molecules_batches, molecules_names_permids), key = lambda item: (item[0], item[1]), reverse=True)]
molecules_names_permids.insert(0, ('Select a molecule...', None))
molecules_dropdown.options = molecules_names_permids
molecules_dropdown.options = molecules_names_permids

display(parents_boxes)
create_deposition_button.on_click(create_deposition_action)
quit_button.on_click(close_notebook)
samples_dropdown.observe(load_sample_metadata, names = 'value')
molecules_dropdown.observe(load_molecule_metadata, names = 'value')

## Properties

In [ ]:
display(deposition_name_textbox, deposition_properties, comments_textbox)

## Sample out

In [ ]:
deposition_name_textbox.observe(update_text, names = 'value')
display(sample_out_name_textbox)
display(bottom_buttons_hbox)
display(increase_buttons_size)